In [6]:
import torch
import math

In [7]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

/Users/sibivishtan/anaconda3/envs/learning_dl/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/sibivishtan/anaconda3/envs/learning_dl/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [8]:
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

torch_dtype = torch.float16 if device == "mps" else torch.float32


In [5]:
device

'mps'

In [6]:
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())

True
True


In [3]:
import torch

# Check for MPS availability
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS backend is available.")
else:
    device = torch.device("cpu")
    print("MPS backend is not available. Using CPU.")

# Simple tensor operation
tensor = torch.randn((3, 3), device=device)
print(f"Tensor on {device}: {tensor}")

# Simple model to check if training works
model = torch.nn.Linear(3, 3).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()

# Dummy data
data = torch.randn((10, 3), device=device)
target = torch.randn((10, 3), device=device)

# Forward pass
output = model(data)
loss = loss_fn(output, target)

# Backward pass
loss.backward()
optimizer.step()

print(f"Loss: {loss.item()}")


MPS backend is available.
Tensor on mps: tensor([[ 0.3527, -0.6639,  0.0435],
        [-0.2579,  1.0143,  1.4424],
        [ 1.1430,  0.2052,  0.2212]], device='mps:0')
Loss: 1.5051764249801636


In [2]:
!pip install torch --upgrade

In [ ]:
model_id  = "openai/whisper-large-v3"

In [9]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id).to(device=device, dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

TypeError: to() received an invalid combination of arguments - got (use_safetensors=bool, low_cpu_mem_usage=bool, dtype=torch.dtype, device=str, ), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (Tensor tensor, bool non_blocking, bool copy, *, torch.memory_format memory_format)


In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer, 
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s = 15,
    batch_size = 16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device 
)